## **Описание задачи**

Вам нужно провести исследование и написать код, который будет оценивать степень влияния признаков на целевую метрику. В нашем случае, целевая метрика - CTR (кликабельность объявления - чем выше, тем лучше). 

Грубо говоря, вам нужно придумать (или использовать какой-то готовый) алгоритм(-ы), которые покажут - насколько при изменении значении каждого признака меняется значение целевой метрики. 

По итогу, на основании ваших расчетов, мы должны автоматически строить примерно такой график:

![](\images\Example_graph.png)

Здесь мы видим, что при изменении объектов CTR меняется довольно сильно, при изменении главного цвета - тоже довольно сильно, также и при изменении длины текста. Чуть меньше CTR зависит от вспомогательных цветов. Остальные признаки слабо влияют на изменение CTR. 

На данном графике % - некая условная величина, которую мы для себя вывели, она показывает степень влияния того или иного признака на целевую метрику. Сумма всех процентов не обязана равняться 100%.

**Таким образом, задача сводится к построению модели регрессионного анализа, предсказывающей изменение количества кликов при изменении того или иного параметра рекламного объявления, с последующей визуализации этой зависимости (динамический дашборд, показывающий важность каждого из признаков для целевой переменной)**

## Описание признаков

adtext_text - текст в теле объявления

adtext_language - язык исходного текста в теле объявления

adtext_eng - перевод текста на англ

adtext_length - длина текста

adtext_nouns - существительные в тексте

adtext_verbs - глаголы в тексте

adtext_adjs - прилагательные

adtext_contains_cta - содержит ли текст призыв к действию (call to action)

adtext_contains_question - содержит ли текст вопрос 

… - далее идет тоже самое для description (описание объялвения) и headline (заголовок объявления)

cta_type - тип кнопки на объявлении

image_text_present - есть ли на картинке текст

image_text - текст на картинке

image_eng_text - англ перевод текста

image_text_area - объем текста

image_text_length - длина текста

image_language - язык текста

image_text_contains_question - содержит вопрос?

image_text_contains_cta - содерит CTA? 

image_nouns - сущ (массив)

image_verbs - глаголы (массив)

image_adjs - прилаг (массив)

image_phrases - отдельные фразы (массив)

image_color_main - главный цвет

image_color_rest - дополнительные цвета  (массив)

image_proportions - пропорции картинки

image_objects - объекты  (массив)

image_tags - дополнительные объекты  (массив)

image_people_amount - кол-во людей на картинке

image_clip - не берите этот признак в расчет

image_clip_keywords - не берите этот признак в расчет

ctr - значение метрики CTR

In [63]:
# Формируем среду`
import pandas as pd
import numpy as np

import plotly.express as px
import matplotlib.pyplot as plt


In [ ]:
# фиксируем RANDOM_SEED для воспроизводимоcти эксперимента
RANDOM_SEED = 42

## Получение и обзор данных

In [5]:
df = pd.read_excel('data/tagging.xlsx')
df.head()

,adtext_text,adtext_language,adtext_eng,adtext_length,adtext_nouns,adtext_verbs,adtext_adjs,adtext_contains_cta,adtext_contains_question,headline_text,...,image_phrases,image_color_main,image_color_rest,image_proportions,image_objects,image_tags,image_people_amount,image_clip,image_clip_keywords,ctr
0,«This app contains everything that I wanted to...,en,This app contains everything that I wanted to ...,200,"['kind', 'moon', 'spirituality', 'lot', 'calen...","['enjoy', 'introducing', 'wanted', 'contains',...","['favorite', 'different']",False,False,Magic in your pocket ✨👉,...,"['hot apple cider', 'collect acorns', 'and nut...",#ccffff,['#6666cc'],2048x2048,['tableware'],"['poster', 'illustration', 'font', 'product', ...",1,a mulato man with a notebook sitting at the ta...,"['word1', 'word2', 'word3']",0.007162
1,"✨ Follow Moon phases, receive positive affirma...",en,Follow Moon phases receive positive affirmati...,100,"['moon', 'runes', 'affirmations', 'phases', 'a...","['follow', 'learn', 'receive']","['positive', 'moonly']",False,False,Try Moon Calendar 👉,...,"['regret', 'anxiety', 'you', 'action', 'change...",#666666,"['#333333', '#663333', '#999999', '#333300']",2048x2048,[],"['electric blue', 'landscape', 'font', 'slope'...",1,a mulato man with a notebook sitting at the ta...,"['word1', 'word2', 'word3']",0.005659
2,"✨ Follow Moon phases, receive positive affirma...",en,Follow Moon phases receive positive affirmati...,100,"['moon', 'runes', 'affirmations', 'phases', 'a...","['follow', 'learn', 'receive']","['positive', 'moonly']",False,False,Try Moon Calendar 👉,...,"['regret', 'anxiety', 'you', 'action', 'change...",#666666,"['#333333', '#663333', '#999999', '#333300']",2048x2048,[],"['electric blue', 'landscape', 'font', 'slope'...",1,a mulato man with a notebook sitting at the ta...,"['word1', 'word2', 'word3']",0.005659
3,«This app contains everything that I wanted to...,en,This app contains everything that I wanted to ...,200,"['kind', 'moon', 'spirituality', 'lot', 'calen...","['enjoy', 'introducing', 'wanted', 'contains',...","['favorite', 'different']",False,False,Magic in your pocket ✨👉,...,"['guide for little witches', 'n', 'moon calend...",#996633,"['#663333', '#666666', '#996666', '#333333']",1280x1280,['packaged goods'],"['carnivore', 'electric blue', 'small to mediu...",1,a mulato man with a notebook sitting at the ta...,"['word1', 'word2', 'word3']",0.007388
4,«This app contains everything that I wanted to...,en,This app contains everything that I wanted to ...,200,"['kind', 'moon', 'spirituality', 'lot', 'calen...","['enjoy', 'introducing', 'wanted', 'contains',...","['favorite', 'different']",False,False,Magic in your pocket ✨👉,...,"['guide for little witches', 'n', 'moon calend...",#996633,"['#663333', '#666666', '#996666', '#333333']",1280x1280,['packaged goods'],"['carnivore', 'electric blue', 'small to mediu...",1,a mulato man with a notebook sitting at the ta...,"['word1', 'word2', 'word3']",0.006329


In [12]:
df[df.columns[28:]].isna().sum()

image_text_present              0
image_text                      0
image_eng_text                  0
image_text_area                 0
image_text_length               0
image_language                  0
image_text_contains_question    0
image_text_contains_cta         0
image_nouns                     0
image_verbs                     0
image_adjs                      0
image_phrases                   0
image_color_main                0
image_color_rest                0
image_proportions               0
image_objects                   0
image_tags                      0
image_people_amount             0
image_clip                      0
image_clip_keywords             0
ctr                             0
dtype: int64

Правильно подобранные слова текста объявления (заголовков, тела, описания), конечно, должны сильно мотивировать клиента на клик по нему, но в данном проект ограничимся влиянием графических решений рекламы, кроме того в признаках текста достаточно много пропусков, поэтому оставим в данных только те столбцы, которые отвечают за картинку объявления

In [14]:
# Избавляемся от ненужных признаков и выводим информацию о датасете
df = df[df.columns[28:]]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   image_text_present            399 non-null    bool   
 1   image_text                    399 non-null    object 
 2   image_eng_text                399 non-null    object 
 3   image_text_area               399 non-null    int64  
 4   image_text_length             399 non-null    int64  
 5   image_language                399 non-null    object 
 6   image_text_contains_question  399 non-null    bool   
 7   image_text_contains_cta       399 non-null    bool   
 8   image_nouns                   399 non-null    object 
 9   image_verbs                   399 non-null    object 
 10  image_adjs                    399 non-null    object 
 11  image_phrases                 399 non-null    object 
 12  image_color_main              399 non-null    object 
 13  image

В датасете 20 признаков, 14 из которых категориальные, 3 - числовые и 3 - булевые, 1 целевая переменнам (ctr), а также 399 наблюдений

## Анализ и обработка данных

In [15]:
# Проверка на пропуски
df.isnull().sum().sum()

0

**Вывод**
Пропуски в данных отсуствуют

### Анализ признаков

In [45]:
# Числовые признаки (без целевой переменныой - последний столбец)
df.iloc[:, :-1].describe()[1:].T

,mean,std,min,25%,50%,75%,max
image_text_area,11.829574,4.529194,0.0,10.0,10.0,10.0,30.0
image_text_length,172.994987,125.197291,25.0,100.0,150.0,200.0,800.0
image_people_amount,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


In [39]:
# Булевы признаки
df.describe(include='bool')[1:].T

,unique,top,freq
image_text_present,1,True,399
image_text_contains_question,1,False,399
image_text_contains_cta,2,False,364


In [31]:
# Категориальные признаки
df.describe(include='object')[1:].T

,unique,top,freq
image_text,73,Guide for little witches\nn\nMoon Calendar\nRu...,69
image_eng_text,73,Guide for little witches\nn\nMoon Calendar\nRu...,69
image_language,2,en,394
image_nouns,60,"['holidays', 'moon', 'sounds', 'calendar', 'wi...",86
image_verbs,54,[],119
image_adjs,43,"['runic', 'little', 'magical', 'ancient']",114
image_phrases,73,"['guide for little witches', 'n', 'moon calend...",69
image_color_main,26,#333333,168
image_color_rest,70,"['#663333', '#666666', '#996666', '#333333']",83
image_proportions,10,2048x2048,257


In [41]:
df[['image_language', 'image_text_contains_cta']].value_counts(normalize=True)

image_language  image_text_contains_cta
en              False                      0.907268
                True                       0.080201
fr              True                       0.007519
                False                      0.005013
Name: proportion, dtype: float64

In [50]:
df_dup = df[df.duplicated()]
print(df_dup.count())

image_text_present              60
image_text                      60
image_eng_text                  60
image_text_area                 60
image_text_length               60
image_language                  60
image_text_contains_question    60
image_text_contains_cta         60
image_nouns                     60
image_verbs                     60
image_adjs                      60
image_phrases                   60
image_color_main                60
image_color_rest                60
image_proportions               60
image_objects                   60
image_tags                      60
image_people_amount             60
image_clip                      60
image_clip_keywords             60
ctr                             60
dtype: int64


**Выводы**

1. Признаки image_people_amount, image_clip, image_clip_keywords, image_text_present, image_text_contains_question имеют только по одному значению, поэтому никак не влияют на целевую переменную и могут быть удалены
2. Признаки image_language, image_text_contains_cta содержит по два значения, но одно из них сильно превалирует (более 90% наблюдений), поэтому необходимо проверить их влияние на целевую переменную и если оно незначительно, удалить их
4. 60 наблюдений - это дубликаты, которые также подлежат удалению
5. Значение признаков image_nouns, image_verbs, image_adjs, image_phrases, image_color_rest, image_proportions, image_objects, image_tags представляют собой списки (масссивы) значений, в том числе и пустые, которые требуют преобразования
6. В признаке image_text_area всего четыре уникальных знаыения, одно из которых (максимальное) встречается однажды, а следовательно с таким наблюдением тоже можно растаться, признав его выбросом, а сам признак может быть закодироован 



### Анализ целевой переменной

In [88]:
#Построим распределение таргета
fig = px.histogram(df,
                   x='ctr',
                   title='Распределение кликабельности (CTR)'
                   )
fig.show();

**Вывод**


## EDA

In [94]:
#Удаляем неинформативные признаки
drop_col = ['image_people_amount', 'image_clip', 'image_clip_keywords', 'image_text_present', 'image_text_contains_question']
df_drop = df.drop(df[drop_col], axis=1)


In [101]:
# Удаляем дубликаты

df_drop = df_drop.drop_duplicates()
df_drop.shape


(339, 16)

In [104]:
px.histogram(df_drop.image_text_length)

In [117]:
df[df.image_language == 'fr'].image_tags#.duplicated()

5      ['cg artwork', 'magenta', 'purple', 'font', 'c...
154    ['happy', 'magenta', 'font', 'fictional charac...
188    ['illustration', 'happy', 'purple', 'font', 'm...
255    ['eye', 'art', 'cartoon', 'doll', 'eyebrow', '...
256    ['eye', 'art', 'cartoon', 'doll', 'eyebrow', '...
Name: image_tags, dtype: object

In [72]:
df[df.image_text_area < 20]

,image_text_present,image_text,image_eng_text,image_text_area,image_text_length,image_language,image_text_contains_question,image_text_contains_cta,image_nouns,image_verbs,...,image_phrases,image_color_main,image_color_rest,image_proportions,image_objects,image_tags,image_people_amount,image_clip,image_clip_keywords,ctr
0,True,Hot apple cider\nCollect acorns\nand nuts\nMab...,Hot apple cider\nCollect acorns\nand nuts\nMab...,10,150,en,False,False,"['fruit', 'thanks', 'leaves', 'pies', 'cider',...","['reflect', 'give', 'collect']",...,"['hot apple cider', 'collect acorns', 'and nut...",#ccffff,['#6666cc'],2048x2048,['tableware'],"['poster', 'illustration', 'font', 'product', ...",1,a mulato man with a notebook sitting at the ta...,"['word1', 'word2', 'word3']",0.007162
1,True,Regret\nAnxiety\nYou\nAction\nChange your mindset,Regret\nAnxiety\nYou\nAction\nChange your mindset,0,50,en,False,False,"['action', 'anxiety', 'mindset']","['regret', 'change']",...,"['regret', 'anxiety', 'you', 'action', 'change...",#666666,"['#333333', '#663333', '#999999', '#333300']",2048x2048,[],"['electric blue', 'landscape', 'font', 'slope'...",1,a mulato man with a notebook sitting at the ta...,"['word1', 'word2', 'word3']",0.005659
2,True,Regret\nAnxiety\nYou\nAction\nChange your mindset,Regret\nAnxiety\nYou\nAction\nChange your mindset,0,50,en,False,False,"['action', 'anxiety', 'mindset']","['regret', 'change']",...,"['regret', 'anxiety', 'you', 'action', 'change...",#666666,"['#333333', '#663333', '#999999', '#333300']",2048x2048,[],"['electric blue', 'landscape', 'font', 'slope'...",1,a mulato man with a notebook sitting at the ta...,"['word1', 'word2', 'word3']",0.005659
3,True,Guide for little witches\nn\nMoon Calendar\nRu...,Guide for little witches\nn\nMoon Calendar\nRu...,10,100,en,False,False,"['holidays', 'moon', 'sounds', 'calendar', 'wi...",[],...,"['guide for little witches', 'n', 'moon calend...",#996633,"['#663333', '#666666', '#996666', '#333333']",1280x1280,['packaged goods'],"['carnivore', 'electric blue', 'small to mediu...",1,a mulato man with a notebook sitting at the ta...,"['word1', 'word2', 'word3']",0.007388
4,True,Guide for little witches\nn\nMoon Calendar\nRu...,Guide for little witches\nn\nMoon Calendar\nRu...,10,100,en,False,False,"['holidays', 'moon', 'sounds', 'calendar', 'wi...",[],...,"['guide for little witches', 'n', 'moon calend...",#996633,"['#663333', '#666666', '#996666', '#333333']",1280x1280,['packaged goods'],"['carnivore', 'electric blue', 'small to mediu...",1,a mulato man with a notebook sitting at the ta...,"['word1', 'word2', 'word3']",0.006329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,True,"""Being sad spurs from\na negative mindset and ...",Being sad spurs from\na negative mindset and n...,10,200,en,False,True,"['ones', 'mindset', 'emotions', 'spurs', 'othe...","['choose', 'needs', 'benefit', 'worked', 'affe...",...,"['""being sad spurs from', 'a negative mindset ...",#663366,[],2048x2048,[],"['electric blue', 'font', 'astronomical object...",1,a mulato man with a notebook sitting at the ta...,"['word1', 'word2', 'word3']",0.009087
395,True,"""Being sad spurs from\na negative mindset and ...",Being sad spurs from\na negative mindset and n...,10,200,en,False,True,"['ones', 'mindset', 'emotions', 'spurs', 'othe...","['choose', 'needs', 'benefit', 'worked', 'affe...",...,"['""being sad spurs from', 'a negative mindset ...",#663366,[],2048x2048,[],"['electric blue', 'font', 'astronomical object...",1,a mulato man with a notebook sitting at the ta...,"['word1', 'word2', 'word3']",0.009087
396,True,Hot apple cider\nCollect acorns\nand nuts\nMab...,Hot apple cider\nCollect acorns\nand nuts\nMab...,10,150,en,False,False,"['fruit', 'thanks', 'leaves', 'pies', 'cider',...","['reflect', 'give', 'collect']",...,"['hot apple cider', 'collect acorns', 'and nut...",#ccffff,['#6666cc'],2048x2048,['tableware'],"['poster', 'illustration', 'font', 'product', ...",1,a m